# Feature Engineering

The next step is to create features from the raw text so we can train the machine learning models. The steps followed are:

1. **Text Cleaning and Preparation**: cleaning of special characters, downcasing, punctuation signs. possessive pronouns and stop words removal and lemmatization. 
2. **Label coding**: creation of a dictionary to map each category to a code.
3. **Train-test split**: to test the models on unseen data.
4. **Text representation**: use of TF-IDF scores to represent text.

In [21]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np
import matplotlib.pyplot as plt 


First of all we'll load the dataset:

In [22]:
path_df = "C:/Users/asus-pc/Documents/PBA/Tugas Akhir/02. Exploratory Data Analysis/News_dataset.pickle"

with open(path_df, 'rb') as data:
    df = pickle.load(data)

In [23]:
df.head()

,title,news_portal,url,img_url,date,content,tag,area,kota,label,id,News_length
0,"""Sebenarnya Saya Sudah Pesan Orkes, Sudah Kasi...",kompas,https://regional.kompas.com/read/2020/04/01/14...,https://asset.kompas.com/crops/KX0cYSp6XFrs-Jt...,2020-04-01,- Zaenal Abidin harus menggelar resepsi pernik...,"Lombok Tengah, ntb positif covid-19, kasus cor...",nusa tenggara barat,lombok tengah,criticism,1,2689
1,"""Update"" Berkala AI Bikin Alat Deteksi Covid-1...",kompas,https://regional.kompas.com/read/2020/12/30/07...,https://asset.kompas.com/crops/dspupYKGzyRECRL...,2020-12-30,- Alat deteksi Covid-19 lewat embusan napas ya...,"UGM, Yogyakarta, GeNose, alat deteksi covid Ge...",di_yogyakarta,yogyakarta,notification of information,1,3573
2,1.190 Tenaga Kesehatan Bantu Penanganan Covid-...,tribunnews,https://www.tribunnews.com/metropolitan/2020/0...,https://cdn-2.tstatic.net/tribunnews/foto/bank...,2020-09-04,Sebanyak 1.190 profesional lolos seleksi ...,"runing news, Covid-19, tenaga kesehatan, Jakar...",dki jakarta,NaN,donation,1,1089
3,1.262 Orang di Secapa TNI AD Bandung Positif C...,tribunnews,https://www.tribunnews.com/corona/2020/07/10/1...,https://cdn-2.tstatic.net/tribunnews/foto/bank...,2020-07-10,Suasana terkini di Sekolah Calon Perwira TNI ...,"Secapa TNI AD, Secapa AD Bandung, Running News...",jawa barat,bandung,criticism,1,4765
4,"1.379.662 Kasus Covid-19 di Indonesia, PPKM Mi...",kompas,https://nasional.kompas.com/read/2021/03/08/05...,https://asset.kompas.com/crops/29iuLluQH9j1eu_...,2021-03-08,- Pandemi sudah satu tahun melanda Indonesia ....,"vaksinasi, Covid-19, Satuan Tugas Penanganan C...",indonesia,NaN,hoax,1,5017


And visualize one sample news content:

In [24]:
df.loc[1]['title']

'"Update" Berkala AI Bikin Alat Deteksi Covid-19 Karya UGM GeNose Makin Pintar'

## 1. Text cleaning and preparation

### 1.1. Special character cleaning

We can see the following special characters:

* ``\r``
* ``\n``
* ``\`` before possessive pronouns (`government's = government\'s`)
* ``\`` before possessive pronouns 2 (`Yukos'` = `Yukos\'`)
* ``"`` when quoting text

In [25]:
# \r and \n
df['title_parsed_1'] = df['title'].str.replace("\r", " ")
df['title_parsed_1'] = df['title_parsed_1'].str.replace("\n", " ")
df['title_parsed_1'] = df['title_parsed_1'].str.replace("    ", " ")

Regarding 3rd and 4th bullet, although it seems there is a special character, it won't affect us since it is not a *real* character:

In [26]:
text = "Mr Greenspan\'s"
text

"Mr Greenspan's"

In [27]:
# " when quoting text
df['title_parsed_1'] = df['title_parsed_1'].str.replace('"', '')

### 1.2. Upcase/downcase

We'll downcase the texts because we want, for example, `Football` and `football` to be the same word.

In [28]:
# Lowercasing the text
df['title_parsed_2'] = df['title_parsed_1'].str.lower()

### 1.3. Punctuation signs

Punctuation signs won't have any predicting power, so we'll just get rid of them.

In [29]:
punctuation_signs = list("?:!.,;-")
df['title_parsed_3'] = df['title_parsed_2']

for punct_sign in punctuation_signs:
    df['title_parsed_3'] = df['title_parsed_3'].str.replace(punct_sign, '')

By doing this we are messing up with some numbers, but it's no problem since we aren't expecting any predicting power from them.

### 1.4. Possessive pronouns

We'll also remove possessive pronoun terminations:

In [30]:
df['title_parsed_4'] = df['title_parsed_3'].str.replace("'s", "")

### 1.5. Stemming and Lemmatization

Since stemming can produce output words that don't exist, we'll only use a lemmatization process at this moment. Lemmatization takes into consideration the morphological analysis of the words and returns words that do exist, so it will be more useful for us.

In [31]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()


In [32]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['title_parsed_3']
    
    sentence = text
    lemmatized_text = stemmer.stem(sentence)

    print(lemmatized_text)
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

benar saya sudah pesan orkes sudah kasih uang muka nyata gagal
update kala ai bikin alat deteksi covid19 karya ugm genose makin pintar
1190 tenaga sehat bantu tangan covid19 di jakarta hingga akhir tahun 2020
1262 orang di capa tni ad bandung positif corona warga minta tak panik
1379662 kasus covid19 di indonesia ppkm mikro klaim tekan kasus hari
10 tanya awam soal vaksin covid19 dan sakit yang milik ini jawab idi
100 ventilator canggih bantu dari amerika serikat tiba di indonesia
11 bulan pandemi covid19 perintah minta baik komunikasi publik
12 karyawan pertamina odp covid19 usir dari karantina anggap tak patuh ikut fakta
12 ton alat sehat yang baru tiba di indonesia bantu perintah tiongkok
120 juta orang miskin terima bantu rp 72 triliun saat pandemi covid19
13 hari isolasi mandiri wagub dki diet gagal total hari 5 kali makan banyak dapat kirim jamu
18 klub liga 1 minta guna bus dirut pt lib ungkap alas tidak guna pesawat
2 alumni ui produksi baju hazmat untuk tenaga medis awal pakai

anggota dpr minta kemenkes beri klarifikasi soal beri vaksin covid19 untuk helena lim
anggota dpr ri suara potong gaji dalam rapat paripurna untuk bantu tangan virus corona
anggota dpr sebut tunggak klaim biaya awat pasien covid19 karena masalah administrasi
anggota dprd bantul dapat keranda mayat telah sama makam covid19 seperti anjing
anggota dprd bantul sebut pasien laku positif covid dan makam seperti anjing plh bupati enggak pas
anggota dprd bantul sebut makam covid19 cuma proyek rawan geruduk dan tuntut klarifikasi
anggota dprd bengkulu hingga pns di lumajang sumbang gaji bantu tangan covid19
anggota dprd jakarta ini akan sumbang gaji untuk tangan corona ajak kader lain ikut aksi
anggota dprd tanya letak lahan makam yang beli pemprov dki harga rp 185 m
anggota dprd hanya gubernur dki yang mau lepas tanggung jawab tangan covid19
anggota komisi i dpr singgung peran arif lokal dalam tangkal disinformasi
anggota marah saat minta cek sehat wakil ketua dprd blora minta maaf ini jelas
a

bbplk serang donasi alat cegah covid19
bbplk serang sumbang apd untuk tangan covid19 di cilegon
bea cukai kian aktif salur donasi apd untuk tenaga sehat di bagai kota di indonesia
beban presiden jokowi berat hadap covid19 menko pmk minta lebih peran
ajar dari korea selatan indonesia minta tak terburuburu terap new normal
ajar dari pasien covid19 yang sembuh ini aja wagub ntt tanggap hoaks corona
belanja online produk lokal umkm ikut bantu tingkat ekonomi indonesia
beli kakao dari tani indonesia keiichi sumbang cokelat produk ke kerja medis di kyoto jepang
benar kasus covid19 di dki jakarta lambat cek fakta grafik kembang sejak psbb
benar perintah lebih penting ekonomi banding sehat ini jawab menteri sri mulyani
benar virus corona tular lewat udara tatap mata mata najwa bongkar horor 60 hoaks covid19
bendung katulampa siaga ii warga bantaran kali ciliwung minta waspada banjir
bentuk kebal komunitas masyarakat minta tak ragu ikut vaksinasi covid19
awal lihat unggah teman di medsos siswi 

breaking news update corona indonesia 29 juli tambah 2381 pasien total 104432 kasus positif
breaking news update corona indonesia 3 april total 1986 positif 134 sembuh 181 tinggal
breaking news update corona indonesia 3 januari 2021 tambah 6877 kasus total 765350 positif
breaking news update corona indonesia 30 desember 2020 tambah 8002 kasus total 735124 positif
breaking news update corona indonesia 30 mei tambah 557 pasien total 25773 kasus positif
breaking news update corona indonesia 31 januari 2021 tambah 12001 kasus total 1078314 positif
breaking news update corona indonesia 31 juli tambah 2040 pasien total 108376 kasus positif
breaking news update corona indonesia 31 oktober 2020 tambah 3143 kasus total 410088 positif
breaking news update corona indonesia 4 mei kasus positif tambah 395 total jadi 11587
breaking news update corona indonesia 5 mei tambah 484 kasus baru pasien positif jadi 12071
breaking news update corona indonesia 5 oktober 2020 tambah 3622 kasus total 307120 pos

di jatim pasien covid19 yang sembuh lebih banyak ketimbang yang tinggal ikut updatenya
di tengah pandemi corona psi kritik pemprov dki bayar commitment fee formula e rp 200 m
di tengah pandemi aparat minta beri akses optimal kepada kuasa hukum temu klien
di tengah skandal vaksin menkes argentina minta presiden undur diri
anggap gagal tangan ekonomi pm thailand prayut hadap mosi tidak percaya
anggap hina ini klarifikasi profesor harvard soal virus corona di indonesia
anggap terlalu dini 5 tokoh ini tak tuju perintah terap new normal fadli zon beri kritik
bantu istri pdp di tegal nekat kabur dari ruang isolasi aku bosan kini sudah kembali ke rs
damping pratikno dan teten masduki jokowi ikan bantu modal kerja kepada dagang kecil
dana perintah pemda minta segera bentuk posko covid19 desa
duga tular dari suami 89 ibu hamil di wonogiri positif covid19 kok bisa ini fakta
gelar di vila acara ulang tahun wali kota bekas bubar ini faktafaktanya
kabar lapar daluna dan rahima kini terima bantu dar

...

update kasus corona di indonesia sabtu 3 10 2020 total 299506 kasus positif 225052 sembuh
update kasus corona di indonesia sabtu 31 10 2020 total 410088 kasus positif 337801 sembuh
update kasus corona di indonesia sabtu 5 9 2020 total 190665 kasus positif 136401 sembuh
update kasus corona di indonesia sabtu 8 8 2020 total 123503 kasus positif 79306 sembuh
update kasus corona di indonesia selasa 1 9 2020 total 177571 kasus positif 128057 sembuh
update kasus corona di indonesia selasa 14 7 2020 total 78572 kasus positif 37636 sembuh
update kasus corona di indonesia selasa 20 10 2020 368842 kasus positif 293653 sembuh
update kasus corona di indonesia selasa 22 9 2020 252923 kasus positif 184298 sembuh
update kasus corona di indonesia selasa 27 10 2020 total 396454 kasus positif 322248 sembuh
update kasus corona di indonesia selasa 29 9 2020 total 282724 kasus positif 210437 sembuh
update kasus corona di indonesia selasa 3 11 2020 total 418375 kasus positif 349497 sembuh
update kasus coron

update kasus covid19 di indonesia jumat 25 12 2020 total 700097 kasus positif 570304 sembuh
update kasus covid19 di indonesia per 2 juni 27549 positif corona 7935 sembuh 1663 tinggal
update kasus covid19 di indonesia tambah 102 orang kini total pasien sembuh 548 orang
update kasus covid19 di indonesia tambah 862 kini total kasus positif corona 45891 orang
update kasus covid19 di indonesia saat ini catat ada 193571 odp dan 17754 pdp
update kasus covid19 di indonesia saat ini catat ada 195948 odp dan 18283 pdp
update kasus covid19 di jakarta hari ini total 897 kasus 52 pasien sembuh 90 tinggal
update kasus covid19 di jateng per 20 maret 2000 orang lebih status odp
update kasus covid19 di seluruh indonesia selasa 24 maret 2020 686 konfirmasi positif
update kasus covid19 di sumsel 285 odp 17 orang di antara isolasi
update kasus covid19 indonesia 30 desember kasus baru 8002 sembuh 6958 dan tinggal 241
update kasus covid19 indonesia usul lockdown akhir pekan kasus lampau india wisatawan anjl

update virus corona di bahan dunia 131627 kasus 4940 tinggal 68529 sembuh
update virus corona di dunia 112 juta kasus anggur di inggris tingkat imbas lockdown
update virus corona di dunia tembus 168 negara 107247 sembuh 18612 tinggal
update virus corona di indonesia tambah 218 orang positif covid19 pasien jadi 2965 kasus
update virus corona di indonesia per 2 mei 2020 total ada 10843 kasus sembuh jadi 1665
update virus corona di indonesia per 30 april 2020 total 10118 kasus positif sembuh 1522 orang
update virus corona di indonesia rabu 24 juni 2020 total 49009 kasus positif 19658 sembuh
update virus corona di indonesia sabtu 12 12 2020 tambah 6388 total 611631 kasus positif
update virus corona di indonesia sabtu 28 maret 2020 1155 orang positif angka mati tingkat
update virus corona di indonesia 1285 pasien positif di 30 provinsi minta karantina wilayah
update virus corona di indonesia 2 balita nyata positif jumlah pasien sembuh ikut tambah
update virus corona di indonesia 117 infeksi

update pasien positif covid19 di kalsel jadi 5 orang odp 1162
update pasien positif covid19 di kota padang tambah 1 orang
update pasien positif covid19 di papua tambah jadi 3 orang
update pasien positif virus corona di indonesia tembus 6 ribu ikut rinciannya
update pasien tambah 153 total ada 1046 kasus covid19 di indonesia
update pdp asal mataram yang tinggal pekan lalu positif covid19
update per 30 juni kasus positif dan sembuh pasien covid19 samasama sentuh angka ribu
update peta sebar covid19 di jatim 51 orang positif corona surabaya banyak
update peta sebar covid19 di jatim surabaya banyak positif jangkit corona
update positif corona di jakarta 356 orang 31 pasien tinggal
update progres fasilitas isolasi pulau galang 92 persen tuntas 5 april
update rincian kasus virus corona di 28 provinsi di indonesia dan upaya perintah
update sebar 10365 kasus baru covid19 dki tinggi dengan 2563 kasus
update sebar 11557 kasus baru covid19 di 34 provinsi dki tinggi
update sebar 19 pasien covid19 

warga depok tanya halalharam vaksin covid19
warga di sleman cipta masker transparan untuk bantu komunikasi teman tuli dan bisu
warga minta tak besuk bayi 4 bulan yang baru sembuh dari covid19
warga minta tak provokasi kasus youtuber prank transgender kini tangan polrestabes bandung
warga minta tetap di rumah kerja hari akan beri bantu oleh pemprov dki
warga minta tetap disiplin terap prokes meski vaksinasi covid19 mulai
warga minta waspada ampo modus semprot disinfektan virus corona
warga jabar yang hasil dampak covid19 dapat bantu rp 500000
warga medan yang ikut ke yerusalem dan italia minta cek sehat
warga pademangan minta tak perlu khawatir tidak bagi vaksin covid19
warga telanjur mudik perangkat desa minta sedia fasilitas isolasi
wartawan telah terima vaksin covid19 masyarakat tidak usah takut stop dengar hoaks
waspada tipu mengatasnamakan bnpb galang donasi untuk tangan covid 19 di media sosial
who tak saran bukti vaksin covid19 untuk syarat jalan internasional
who tak wajib vaksi

In [33]:
df['title_parsed_4'] = lemmatized_text_list
df['title_parsed_4']

0       benar saya sudah pesan orkes sudah kasih uang ...
1       update kala ai bikin alat deteksi covid19 kary...
2       1190 tenaga sehat bantu tangan covid19 di jaka...
3       1262 orang di capa tni ad bandung positif coro...
4       1379662 kasus covid19 di indonesia ppkm mikro ...
                              ...                        
3144    update 21 januari jakarta catat 3151 kasus bar...
3145    update covid19 di jakarta 1151 positif 64 semb...
3146    update covid19 di tangsel 34 positif 6 orang t...
3147    korea selatan gagal upaya korea utara retas da...
3148              polri tangan 77 kasus hoaks soal corona
Name: title_parsed_4, Length: 3149, dtype: object

Although lemmatization doesn't work perfectly in all cases (as can be seen in the example below), it can be useful.

### 1.6. Stop words

In [34]:
# Downloading the stop words list
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\asus-
[nltk_data]     pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [35]:
STOPWORDS= stopwords.words('indonesian')
STOPWORDS.extend(['covid','covid-19','covid19,','korona','2020','corona', 'indonesia','2021'])
stop_words = list(STOPWORDS) #membuang kata yang tidak digunakan
print(stop_words)

['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasanya', 'baik', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya', 'berapa', 'berapakah', 'berapalah', 'berapapun', 'berarti', 'berawal', 'berbagai', 'berdatangan', 'beri', 'berikan', 'berikut', 'berikutnya', 'berjumlah', 'berkali-kali', 'berkata', 'berkehendak', 'berkeinginan'

In [36]:
stop_words[0:10]

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir']

To remove the stop words, we'll handle a regular expression only detecting whole words, as seen in the following example:

In [37]:
df['title_parsed_5'] = df['title_parsed_4']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['title_parsed_5'] = df['title_parsed_5'].str.replace(regex_stopword, '')
df['title_parsed_5'] = df['title_parsed_5'].str.replace("covid", '')

In [38]:
punctuation_signs = list("1234567890")
df['title_parsed_6'] = df['title_parsed_5']

for punct_sign in punctuation_signs:
    df['title_parsed_6'] = df['title_parsed_6'].str.replace(punct_sign, '')


We have some dobule/triple spaces between words because of the replacements. However, it's not a problem because we'll tokenize by the spaces later.

As an example, we'll show an original news article and its modifications throughout the process:

In [39]:
df.loc[4]['title']

'1.379.662 Kasus Covid-19 di Indonesia, PPKM Mikro Diklaim Tekan Kasus Harian'

1. Special character cleaning

In [40]:
df.loc[4]['title_parsed_1']

'1.379.662 Kasus Covid-19 di Indonesia, PPKM Mikro Diklaim Tekan Kasus Harian'

2. Upcase/downcase

In [41]:
df.loc[4]['title_parsed_2']

'1.379.662 kasus covid-19 di indonesia, ppkm mikro diklaim tekan kasus harian'

3. Punctuation signs

In [42]:
df.loc[4]['title_parsed_3']

'1379662 kasus covid19 di indonesia ppkm mikro diklaim tekan kasus harian'

4. Possessive pronouns

In [43]:
df.loc[4]['title_parsed_4']

'1379662 kasus covid19 di indonesia ppkm mikro klaim tekan kasus hari'

5. Stemming and Lemmatization

In [44]:
df.loc[4]['title_parsed_5']

'1379662  19   ppkm mikro klaim tekan  '

6. Stop words

In [45]:
df.loc[4]['title_parsed_5']

'1379662  19   ppkm mikro klaim tekan  '

Finally, we can delete the intermediate columns:

In [46]:
df.head(1)

,title,news_portal,url,img_url,date,content,tag,area,kota,label,id,News_length,title_parsed_1,title_parsed_2,title_parsed_3,title_parsed_4,title_parsed_5,title_parsed_6
0,"""Sebenarnya Saya Sudah Pesan Orkes, Sudah Kasi...",kompas,https://regional.kompas.com/read/2020/04/01/14...,https://asset.kompas.com/crops/KX0cYSp6XFrs-Jt...,2020-04-01,- Zaenal Abidin harus menggelar resepsi pernik...,"Lombok Tengah, ntb positif covid-19, kasus cor...",nusa tenggara barat,lombok tengah,criticism,1,2689,"Sebenarnya Saya Sudah Pesan Orkes, Sudah Kasih...","sebenarnya saya sudah pesan orkes, sudah kasih...",sebenarnya saya sudah pesan orkes sudah kasih ...,benar saya sudah pesan orkes sudah kasih uang ...,pesan orkes kasih uang muka nyata gagal,pesan orkes kasih uang muka nyata gagal


In [47]:
list_columns = ["title","title_parsed_6", "news_portal", "url", "img_url", "date","content","tag","area","kota","label"]
df2 = df[list_columns]

df2 = df2.rename(columns={'title_parsed_6': 'title_parsed'})

In [48]:
df2.head()

,title,title_parsed,news_portal,url,img_url,date,content,tag,area,kota,label
0,"""Sebenarnya Saya Sudah Pesan Orkes, Sudah Kasi...",pesan orkes kasih uang muka nyata gagal,kompas,https://regional.kompas.com/read/2020/04/01/14...,https://asset.kompas.com/crops/KX0cYSp6XFrs-Jt...,2020-04-01,- Zaenal Abidin harus menggelar resepsi pernik...,"Lombok Tengah, ntb positif covid-19, kasus cor...",nusa tenggara barat,lombok tengah,criticism
1,"""Update"" Berkala AI Bikin Alat Deteksi Covid-1...",update ai bikin alat deteksi karya ugm genos...,kompas,https://regional.kompas.com/read/2020/12/30/07...,https://asset.kompas.com/crops/dspupYKGzyRECRL...,2020-12-30,- Alat deteksi Covid-19 lewat embusan napas ya...,"UGM, Yogyakarta, GeNose, alat deteksi covid Ge...",di_yogyakarta,yogyakarta,notification of information
2,1.190 Tenaga Kesehatan Bantu Penanganan Covid-...,tenaga sehat bantu tangan jakarta,tribunnews,https://www.tribunnews.com/metropolitan/2020/0...,https://cdn-2.tstatic.net/tribunnews/foto/bank...,2020-09-04,Sebanyak 1.190 profesional lolos seleksi ...,"runing news, Covid-19, tenaga kesehatan, Jakar...",dki jakarta,NaN,donation
3,1.262 Orang di Secapa TNI AD Bandung Positif C...,orang capa tni ad bandung positif warga p...,tribunnews,https://www.tribunnews.com/corona/2020/07/10/1...,https://cdn-2.tstatic.net/tribunnews/foto/bank...,2020-07-10,Suasana terkini di Sekolah Calon Perwira TNI ...,"Secapa TNI AD, Secapa AD Bandung, Running News...",jawa barat,bandung,criticism
4,"1.379.662 Kasus Covid-19 di Indonesia, PPKM Mi...",ppkm mikro klaim tekan,kompas,https://nasional.kompas.com/read/2021/03/08/05...,https://asset.kompas.com/crops/29iuLluQH9j1eu_...,2021-03-08,- Pandemi sudah satu tahun melanda Indonesia ....,"vaksinasi, Covid-19, Satuan Tugas Penanganan C...",indonesia,NaN,hoax


**IMPORTANT:**

We need to remember that our model will gather the latest news articles from different newspapers every time we want. For that reason, we not only need to take into account the peculiarities of the training set articles, but also possible ones that are present in the gathered news articles.

For this reason, possible peculiarities have been studied in the *05. News Scraping* folder.

## 2. Label coding

We'll create a dictionary with the label codification:

In [49]:
label_codes = {
    'notification of information': 0,
    'donation': 1,
    'criticism': 2,
    'hoax': 3,
}

In [50]:
# Category mapping
df2['label_code'] = df2['label']
df2 = df2.replace({'label_code':label_codes})

In [51]:
df2.head()

,title,title_parsed,news_portal,url,img_url,date,content,tag,area,kota,label,label_code
0,"""Sebenarnya Saya Sudah Pesan Orkes, Sudah Kasi...",pesan orkes kasih uang muka nyata gagal,kompas,https://regional.kompas.com/read/2020/04/01/14...,https://asset.kompas.com/crops/KX0cYSp6XFrs-Jt...,2020-04-01,- Zaenal Abidin harus menggelar resepsi pernik...,"Lombok Tengah, ntb positif covid-19, kasus cor...",nusa tenggara barat,lombok tengah,criticism,2
1,"""Update"" Berkala AI Bikin Alat Deteksi Covid-1...",update ai bikin alat deteksi karya ugm genos...,kompas,https://regional.kompas.com/read/2020/12/30/07...,https://asset.kompas.com/crops/dspupYKGzyRECRL...,2020-12-30,- Alat deteksi Covid-19 lewat embusan napas ya...,"UGM, Yogyakarta, GeNose, alat deteksi covid Ge...",di_yogyakarta,yogyakarta,notification of information,0
2,1.190 Tenaga Kesehatan Bantu Penanganan Covid-...,tenaga sehat bantu tangan jakarta,tribunnews,https://www.tribunnews.com/metropolitan/2020/0...,https://cdn-2.tstatic.net/tribunnews/foto/bank...,2020-09-04,Sebanyak 1.190 profesional lolos seleksi ...,"runing news, Covid-19, tenaga kesehatan, Jakar...",dki jakarta,NaN,donation,1
3,1.262 Orang di Secapa TNI AD Bandung Positif C...,orang capa tni ad bandung positif warga p...,tribunnews,https://www.tribunnews.com/corona/2020/07/10/1...,https://cdn-2.tstatic.net/tribunnews/foto/bank...,2020-07-10,Suasana terkini di Sekolah Calon Perwira TNI ...,"Secapa TNI AD, Secapa AD Bandung, Running News...",jawa barat,bandung,criticism,2
4,"1.379.662 Kasus Covid-19 di Indonesia, PPKM Mi...",ppkm mikro klaim tekan,kompas,https://nasional.kompas.com/read/2021/03/08/05...,https://asset.kompas.com/crops/29iuLluQH9j1eu_...,2021-03-08,- Pandemi sudah satu tahun melanda Indonesia ....,"vaksinasi, Covid-19, Satuan Tugas Penanganan C...",indonesia,NaN,hoax,3


## 3. Train - test split

We'll set apart a test set to prove the quality of our models. We'll do Cross Validation in the train set in order to tune the hyperparameters and then test performance on the unseen data of the test set.

In [52]:
X_train, X_test, y_train, y_test = train_test_split(df2['title_parsed'], 
                                                    df2['label_code'], 
                                                    test_size=0.15, 
                                                    random_state=8)

Since we don't have much observations (only 2.225), we'll choose a test set size of 15% of the full dataset.

## 4. Text representation

We have various options:

* Count Vectors as features
* TF-IDF Vectors as features
* Word Embeddings as features
* Text / NLP based features
* Topic Models as features

We'll use **TF-IDF Vectors** as features.

We have to define the different parameters:

* `ngram_range`: We want to consider both unigrams and bigrams.
* `max_df`: When building the vocabulary ignore terms that have a document
    frequency strictly higher than the given threshold
* `min_df`: When building the vocabulary ignore terms that have a document
    frequency strictly lower than the given threshold.
* `max_features`: If not None, build a vocabulary that only consider the top
    max_features ordered by term frequency across the corpus.

See `TfidfVectorizer?` for further detail.

It needs to be mentioned that we are implicitly scaling our data when representing it as TF-IDF features with the argument `norm`.

In [53]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

We have chosen these values as a first approximation. Since the models that we develop later have a very good predictive power, we'll stick to these values. But it has to be mentioned that different combinations could be tried in order to improve even more the accuracy of the models.

In [55]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
features_train2 = tfidf.fit_transform(X_train)
sums = features_train2.sum(axis = 0)

labels_train = y_train

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test

Please note that we have fitted and then transformed the training set, but we have **only transformed** the **test set**.

We can use the Chi squared test in order to see what unigrams and bigrams are most correlated with each category:

In [62]:
from sklearn.feature_selection import chi2
import numpy as np

features_train2 = tfidf.fit_transform(X_train)

for Product, label_id in sorted(label_codes.items()):
    features_chi2 = chi2(features_train, labels_train == label_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    
    print("# '{}' label:".format(Product))
#     data = {
#             'Unigram': [unigrams[-1],unigrams[-2],unigrams[-3],unigrams[-4],unigrams[-5]],
#             'Bigram': [bigrams[-1],bigrams[-2],bigrams[-3],"",""]
#            }
#     df3 = pd.DataFrame (data, columns = ['Unigram','Bigram'])
#     print(df3.head())
    print("   Most correlated unigrams:\n {}".format('\n '.join(unigrams[-15:])))
    print("   Most correlated bigrams:\n {}".format('\n '.join(bigrams[-15:])))
    print("")


# 'criticism' label:
   Most correlated unigrams:
 terap
 bijak
 sembuh
 bantu
 dpr
 jokowi
 masyarakat
 ekonomi
 update
 perintah
 gagal
 bukti
 kritik
 singgung
 buruk
   Most correlated bigrams:
 update mei
 update pasien
 anies baswedan
 tinggal dunia
 news update
 breaking news
 pasien sembuh
 update maret
 update januari
 update februari
 pasien positif
 pemprov dki
 sembuh tinggal
 total positif
 positif sembuh

# 'donation' label:
   Most correlated unigrams:
 medis
 positif
 miliar
 alat
 gaji
 apd
 sosial
 update
 rp
 salur
 dampak
 ikan
 donasi
 sumbang
 bantu
   Most correlated bigrams:
 update maret
 update januari
 update februari
 rumah sakit
 sembuh tinggal
 rp juta
 total positif
 positif sembuh
 tenaga medis
 rp miliar
 warga dampak
 terima bantu
 bantu warga
 bantu tangan
 salur bantu

# 'hoax' label:
   Most correlated unigrams:
 tular
 psbb
 awat
 total
 kendali
 viral
 bantu
 obat
 video
 vaksin
 update
 klarifikasi
 hoaks
 klaim
 fakta
   Most correlated bigrams:

In [63]:
len(bigrams)

100

As we can see, the unigrams correspond well to their category. However, bigrams do not. If we get the bigrams in our features:

In [120]:
bigrams

['tenaga sehat',
 'sebar covid',
 'suntik vaksin',
 'kait covid',
 'jawa timur',
 'jawa barat',
 'pasien covid',
 'dki jakarta',
 'rumah sakit',
 'covid jakarta',
 'barat januari',
 'warga dampak',
 'kalsel januari',
 'ridwan kamil',
 'bantu warga',
 'wali kota',
 'covid sembuh',
 'covid jabar',
 'satgas covid',
 'terima bantu',
 'rp miliar',
 'sulteng sultra',
 'sultra malu',
 'malu malut',
 'malut papua',
 'rp juta',
 'presiden jokowi',
 'papua papua',
 'covid sulteng',
 'anies baswedan',
 'bantu tangan',
 'covid tinggal',
 'sumsel babel',
 'jateng banten',
 'babel lampung',
 'banten sumsel',
 'pasien tinggal',
 'orang positif',
 'jabar jateng',
 'orang tinggal',
 'papua barat',
 'salur bantu',
 'update dki',
 'tenaga medis',
 'covid orang',
 'juni total',
 'covid aceh',
 'update sebar',
 'update desember',
 'update dunia',
 'pemprov dki',
 'mei total',
 'maret positif',
 'update september',
 'jambi bengkulu',
 'kepri jambi',
 'riau kepri',
 'sumut sumbar',
 'sumbar riau',
 'aceh sum

We can see there are only six. This means the unigrams have more correlation with the category than the bigrams, and since we're restricting the number of features to the most representative 300, only a few bigrams are being considered.

Let's save the files we'll need in the next steps:

In [155]:
# X_train
with open('Pickles_title/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open('Pickles_title/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open('Pickles_title/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open('Pickles_title/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open('Pickles_title/df.pickle', 'wb') as output:
    pickle.dump(df2, output)
    
# features_train
with open('Pickles_title/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('Pickles_title/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('Pickles_title/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('Pickles_title/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open('Pickles_title/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)